# Deep Learning for NLP - Project

RULES:

* Do not create any additional cell

* Fill in the blanks

* All cells should be runnable (modulo trivial compatibility bugs that we'd fix)

* 4 / 20 points will be allocated to the clarity of your code

* Efficient code will have a bonus

DELIVERABLE:

* the pdf with your answers
* this notebook
* the predictions of the SST test set

DO NOT INCLUDE THE DATASETS IN THE DELIVERABLE..

In [0]:
# Python 3.6 or above is required
from collections import defaultdict
import gzip
import numpy as np
from pathlib import Path
from urllib.request import urlretrieve
import re 
from collections import defaultdict

In [0]:
import os
try: 
  os.mkdir('./data')
except  OSError :
  pass
PATH_TO_DATA = Path('data/')
# Download word vectors, might take a few minutes and about ~3GB of storage space
en_embeddings_path = PATH_TO_DATA / 'cc.en.300.vec.gz'
if not en_embeddings_path.exists():
    urlretrieve('https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.vec.gz', en_embeddings_path)
fr_embeddings_path = PATH_TO_DATA / 'cc.fr.300.vec.gz'
if not fr_embeddings_path.exists():
    urlretrieve('https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.fr.300.vec.gz', fr_embeddings_path)

# 1) Monolingual (English) word embeddings 

In [0]:
class Word2Vec():
    def __init__(self, filepath, vocab_size=50000):
        self.words, self.embeddings = self.load_wordvec(filepath, vocab_size)
        # Mappings for O(1) retrieval:
        self.word2id = {word: idx for idx, word in enumerate(self.words)}
        self.id2word = {idx: word for idx, word in enumerate(self.words)}
    
    def load_wordvec(self, filepath, vocab_size):
        assert str(filepath).endswith('.gz')
        words = []
        embeddings = []
        with gzip.open(filepath, 'rt',encoding="utf8") as f:  # Read compressed file directly
            next(f)  # Skip header
            for i, line in enumerate(f):
                word, vec = line.split(' ', 1)
                words.append(word)
                embeddings.append(np.fromstring(vec, sep=' '))
                if i == (vocab_size - 1):
                    break
        print('Loaded %s pretrained word vectors' % (len(words)))
        return words, np.vstack(embeddings)
    
    def encode(self, word):
        
        # Returns the 1D embedding of a given word
        try:
            return self.embeddings[self.word2id[word]]
        except KeyError:
            #print("word ",word, " not in vocab")
            return np.zeros(self.embeddings[0].shape)
        
    
    def score(self, word1, word2):
        # Return the cosine similarity: use np.dot & np.linalg.norm
        v1=self.encode(word1)
        v2=self.encode(word2)
        return (v1.T@v2)/(np.linalg.norm(v1)*np.linalg.norm(v2))
    
    def most_similar(self, word, k=5):
        # Returns the k most similar words: self.score & np.argsort
        scores=[self.score(word,vocab) for vocab in self.words]
        sorted_scores=np.flip(np.argsort(scores,axis=None))
        return [self.id2word[idx] for idx in sorted_scores[1:k+1]]

In [10]:
word2vec = Word2Vec(en_embeddings_path, vocab_size=50000)
# You will be evaluated on the output of the following:
for word1, word2 in zip(('cat', 'cat', 'cat', 'Paris', 'Paris', 'Paris', 'Paris'), ('tree', 'dog', 'pet', 'France', 'Germany', 'baguette', 'donut')):
    print(word1, word2, word2vec.score(word1, word2))
for word in ['cat', 'dog', 'dogs', 'Paris', 'Germany']:
    print(word2vec.most_similar(word))

Loaded 50000 pretrained word vectors
cat tree 0.2644975466165475
cat dog 0.7078641298542562
cat pet 0.6753313359976381
Paris France 0.6892958925806542
Paris Germany 0.4051242286737548
Paris baguette 0.29399958277802224
Paris donut -0.006588507552348005
['cats', 'kitty', 'kitten', 'feline', 'dog']
['dogs', 'puppy', 'pup', 'canine', 'pet']
['dog', 'cats', 'puppies', 'Dogs', 'pets']
['France', 'Parisian', 'Marseille', 'Brussels', 'Strasbourg']
['Austria', 'Europe', 'Berlin', 'Hamburg', 'Bavaria']


In [0]:
class BagOfWords():
    
    def __init__(self, word2vec):
        self.word2vec = word2vec
    
    def build_idf(self, sentences):
        # build the idf dictionary: associate each word to its idf value
        # -> idf = {word: idf_value, ...}
        occ=defaultdict(float)
        for sentence in sentences:
            words=np.unique([word for word in re.sub(r'\W+', ' ', sentence).split()])
            for word in words:
                occ[word]+=1
        D=len(sentences)
        idf={k:np.log(D/v) for k,v in occ.items()}
        return idf
    
    def encode(self, sentence, idf=None):
        # Takes a sentence as input, returns the sentence embedding
        # mean of word vectors
        words=[word for word in re.sub(r'\W+', ' ', sentence).split()]
        if idf is None:
            words=[self.word2vec.encode(word) for word in words]
            return np.average(words,axis=0)
        else:
            # idf-weighted mean of word vectors
            words=[self.word2vec.encode(word)*idf[word] for word in words]
            return np.average(words,axis=0)

    def score(self, sentence1, sentence2, idf=None):
        # Return the cosine similarity: use np.dot & np.linalg.norm
        v1=self.encode(sentence1,idf)
        v2=self.encode(sentence2,idf)
        return (v1.T@v2)/(np.linalg.norm(v1)*np.linalg.norm(v2))
    def most_similar(self, sentence, sentences, idf=None, k=5):
        # Return most similar sentences
        query = self.encode(sentence, idf)
        keys = np.vstack([self.encode(sentence, idf) for sentence in sentences])
        scores=[self.score(sentence,vocab,idf) for vocab in sentences]
        sorted_scores=np.flip(np.argsort(scores,axis=None))
        return [sentences[idx] for idx in sorted_scores[1:k+1]]

In [12]:
word2vec = Word2Vec(en_embeddings_path, vocab_size=2000000)
sentence2vec = BagOfWords(word2vec)

# Load sentences in "PATH_TO_DATA/sentences.txt"
filepath = PATH_TO_DATA / 'sentences.txt'
with open(filepath, 'r') as f:
    sentences = [line.strip('\n') for line in f]


# You will be evaluated on the output of the following:
print('\n\tAverage of word embeddings')
sentence1 = sentences[7]
sentence2 = sentences[13]
print(sentence1)
print(sentence2)
print(sentence2vec.score(sentence1, sentence2))
sentence = sentences[10]
similar_sentences = sentence2vec.most_similar(sentence, sentences)  # BagOfWords-mean
print(sentence)

for i, sentence in enumerate(similar_sentences):
    print(str(i+1) + ')', sentence)
    
# Build idf scores for each word
idf = sentence2vec.build_idf(sentences)


print('\n\tidf weighted average of word embeddings')
print(sentence1)
print(sentence2)
print(sentence2vec.score(sentence1, sentence2, idf))
sentence = sentences[10]
similar_sentences = sentence2vec.most_similar(sentence, sentences, idf)  # BagOfWords-idf
print(sentence)
for i, sentence in enumerate(similar_sentences):
    print(str(i+1) + ')', sentence)

Loaded 2000000 pretrained word vectors

	Average of word embeddings
1 man singing and 1 man playing a saxophone in a concert . 
10 people venture out to go crosscountry skiing . 
0.6405950330419073
1 smiling african american boy . 
1) 2 woman dancing while pointing . 
2) 5 women and 1 man are smiling for the camera . 
3) 3 males and 1 woman enjoying a sporting event 
4) 2 chinese people wearing traditional clothes 
5) a young boy and 2 girls open christmas presents . 

	idf weighted average of word embeddings
1 man singing and 1 man playing a saxophone in a concert . 
10 people venture out to go crosscountry skiing . 
0.6369156540016581
1 smiling african american boy . 
1) 1 man singing and 1 man playing a saxophone in a concert . 
2) two women and 1 man walking across the street . 
3) 2 guys facing away from camera , 1 girl smiling at camera with blue shirt , 1 guy with a beverage with a jacket on . 
4) 3 males and 1 woman enjoying a sporting event 
5) 5 women and 1 man are smiling fo

# 2) Multilingual (English-French) word embeddings

Let's consider a bilingual dictionary of size V_a (e.g French-English).

Let's define **X** and **Y** the **French** and **English** matrices.

They contain the embeddings associated to the words in the bilingual dictionary.

We want to find a **mapping W** that will project the source word space (e.g French) to the target word space (e.g English).

Procrustes : **W\* = argmin || W.X - Y ||  s.t  W^T.W = Id**
has a closed form solution:
**W = U.V^T  where  U.Sig.V^T = SVD(Y.X^T)**

In what follows, you are asked to: 

In [0]:
class MultilingualWordAligner:
    
    def __init__(self, fr_word2vec, en_word2vec):
        self.fr_word2vec = fr_word2vec
        self.en_word2vec = en_word2vec
        self.aligned_fr_embeddings = self.get_aligned_fr_embeddings()
        
    def get_aligned_fr_embeddings(self):
        # 1 - Get words that appear in both vocabs (= identical character strings)
        identical=[]
        for word in self.fr_word2vec.words:
            if word in self.en_word2vec.word2id: #this is in O(1) in a dict 
                identical+=[word]
        X,Y=[],[]
        for word in identical:
            X+=[self.fr_word2vec.encode(word)]
            Y+=[self.en_word2vec.encode(word)]
        X=np.vstack(X).T
        Y=np.vstack(Y).T
        #     Use it to create the matrix X (emb_dim, vocab_size) and Y (emb_dim, vocab_size) (of embeddings for these words)
        assert X.shape[0] == 300 and Y.shape[0] == 300
        
        # 2 - Solve the Procrustes using the numpy package and: np.linalg.svd() and get the optimal W
        U,_,Vh=np.linalg.svd(Y@X.T)
        W=U@Vh
        #     Now self.fr_word2vec.embeddings * W.transpose() is in the same space as en_word2vec.embeddings
        assert W.shape == (300, 300)
        return np.matmul(fr_word2vec.embeddings, W.transpose())
        
    def get_closest_english_words(self, fr_word, k=3):
        # 3 - Return the top k English nearest neighbors to the input French word
        en_embed=self.aligned_fr_embeddings[self.fr_word2vec.word2id[fr_word]]
        #we'll add the French Word to The English Vocab along with its "english embedding"
        vocab_size=len(self.en_word2vec.id2word)
        self.en_word2vec.embeddings=np.vstack((self.en_word2vec.embeddings,en_embed))
        self.en_word2vec.word2id[fr_word]=vocab_size
        self.en_word2vec.id2word[vocab_size]=fr_word # although we won't need this for the closest words
        return self.en_word2vec.most_similar(fr_word,k)

In [14]:
fr_word2vec = Word2Vec(fr_embeddings_path, vocab_size=50000)
en_word2vec = Word2Vec(en_embeddings_path, vocab_size=50000)
multilingual_word_aligner = MultilingualWordAligner(fr_word2vec, en_word2vec)

# You will be evaluated on the output of the following:
fr_words = ['chat', 'chien', 'voiture', 'zut']
k = 3
for fr_word in fr_words:
    print('-' * 10)
    print(f'fr: "{fr_word}"')
    en_words = multilingual_word_aligner.get_closest_english_words(fr_word, k=3)
    for en_word in en_words:
        print(f'en: "{en_word}"')

Loaded 50000 pretrained word vectors
Loaded 50000 pretrained word vectors
----------
fr: "chat"
en: "cat"
en: "kitten"
en: "kitty"
----------
fr: "chien"
en: "dog"
en: "cat"
en: "pet"
----------
fr: "voiture"
en: "vehicle"
en: "automobile"
en: "motorbike"
----------
fr: "zut"
en: "Ah"
en: "ah"
en: "nope"


If you want to dive deeper on this subject: https://github.com/facebookresearch/MUSE

# 3) Sentence classification with BoV and scikit-learn

In [0]:
# 1 - Load train/dev/test of Stanford Sentiment TreeBank (SST)
#     (https://nlp.stanford.edu/~socherr/EMNLP2013_RNTN.pdf)
train_filepath = PATH_TO_DATA / 'SST/stsa.fine.train'
dev_filepath = PATH_TO_DATA / 'SST/stsa.fine.dev'
test_filepath = PATH_TO_DATA / 'SST/stsa.fine.test.X'
def create_dataset(path,test=False):
    loader = open(path,"r")
    X=[]
    y=[]
    for sentence in loader:
        sentence=re.sub(r'\W+', ' ', sentence)
        if test:
            X+=[sentence.strip('\n')]
        else:
            y+=[int(sentence[0])]
            X+=[sentence[1:].strip('\n')]
    loader.close()
    if test:
        return X
    else:
        return X,y
train_s,y_train=create_dataset(train_filepath)
val_s,y_val=create_dataset(dev_filepath)
test_s=create_dataset(test_filepath,test=True)


In [0]:
# 2 - Encode sentences with the BoV model above
#we'll Use idf weights as this seems to improve performance
idf = sentence2vec.build_idf(np.concatenate((train_s,val_s,test_s)))
def encode(sentences,BoV,idf=None):
    embeddings=[]
    for sentence in sentences:
        embeddings+=[BoV.encode(sentence,idf)]
    return np.vstack(embeddings)
X_train=encode(train_s,sentence2vec,idf)
X_val=encode(val_s,sentence2vec,idf)
X_test=encode(test_s,sentence2vec,idf)

In [17]:
# 3 - Learn Logistic Regression on top of sentence embeddings using scikit-learn
#     (consider tuning the L2 regularization on the dev set)
#     In the paper, the accuracy for average of word vectors is 32.7%
#     (VecAvg, table 1, https://nlp.stanford.edu/~socherr/EMNLP2013_RNTN.pdf)

#we'll just use cross Validation here but we could've also tried using the score on the dev set as mentioned
from sklearn.linear_model import LogisticRegression,LogisticRegressionCV
from sklearn.metrics import classification_report
Cs=np.linspace(3,4,20) # these values are chosen after first testing over larger increments 
clf = LogisticRegressionCV(Cs=Cs,multi_class='multinomial',solver='lbfgs',max_iter=1000,n_jobs=-1,cv=5)
clf.fit(X_train, y_train)
print(classification_report(y_val,clf.predict(X_val)))

              precision    recall  f1-score   support

           0       0.34      0.19      0.24       139
           1       0.43      0.55      0.48       289
           2       0.35      0.18      0.24       229
           3       0.40      0.58      0.47       279
           4       0.50      0.39      0.44       165

    accuracy                           0.41      1101
   macro avg       0.40      0.38      0.37      1101
weighted avg       0.40      0.41      0.39      1101



In [0]:
# 4 - Produce 2210 predictions for the test set (in the same order). One line = one prediction (=0,1,2,3,4).
#     Attach the output file "logreg_bov_y_test_sst.txt" to your deliverable.
#     You will be evaluated on the results of the test set.
pred=[str(i)+'\n' for i in clf.predict(X_test)]
file = open('logreg_bow_y_test_sst.txt',"w")
file.writelines(pred)
file.close()

In [19]:
# BONUS!
# 5 - Try to improve performance with another classifier
#we'll use XGBoost
#     Attach the output file "XXX_bov_y_test_sst.txt" to your deliverable (where XXX = the name of the classifier)
# XGBoost 
import xgboost as xgb
clf = xgb.XGBClassifier(n_jobs=12,max_depth = 3,n_estimators = 290) #Parameters chosen after doing cross validation 
clf.fit(X_train,y_train)
print(classification_report(y_val,clf.predict(X_val)))
#Outputting Prediction 
pred=[str(i)+'\n' for i in clf.predict(X_test)]
file = open('XXX_bow_y_test_sst.txt',"w")
file.writelines(pred)
file.close()

              precision    recall  f1-score   support

           0       0.48      0.23      0.31       139
           1       0.41      0.55      0.47       289
           2       0.45      0.19      0.27       229
           3       0.40      0.61      0.48       279
           4       0.48      0.36      0.41       165

    accuracy                           0.42      1101
   macro avg       0.44      0.39      0.39      1101
weighted avg       0.43      0.42      0.40      1101



# 4) Sentence classification with LSTMs in Keras

## 4.1 - Preprocessing

In [35]:
# 1 - Using the same dataset, transform text to integers using tf.keras.preprocessing.text.one_hot function
#     https://keras.io/preprocessing/text/
import tensorflow as tf
import torch
from torch.utils.data import TensorDataset, DataLoader
import nltk
nltk.download('punkt')
from collections import Counter
def Tokenize(sentences,words=None):
    for i, sentence in enumerate(sentences):
        #The sentences will be stored as a list of words/tokens
        sentences[i] = []
        for word in nltk.word_tokenize(sentence): #Tokenizing the words
            if words!=None:
                words.update([word.lower()]) #Converting all the words to lower case
            sentences[i].append(word)
    return sentences

def one_hot(sentences):
    encoding=[]
    for sentence in sentences:
        encoding+=[torch.Tensor(tf.keras.preprocessing.text.one_hot(sentence,19538))]#19538 is the total number of words in dataset 
    return encoding 


words = Counter() #Dictionary that will map a word to the number of times it appeared in all the training sentences
train_seq=Tokenize(train_s,words)
val_seq=Tokenize(val_s)
test_seq=Tokenize(test_s)

# Removing the words that only appear once
words = {k:v for k,v in words.items() if v>1}
# Sorting the words according to the number of appearances, with the most common word being first
words = sorted(words, key=words.get, reverse=True)
# Adding padding and unknown to our vocabulary so that they will be assigned an index
words = ['_PAD','_UNK'] + words
# Dictionaries to store the word to index mappings and vice versa
word2idx = {o:i for i,o in enumerate(words)}
idx2word = {i:o for i,o in enumerate(words)}

def one_hot(sentences,word2idx,unknown_index=1):
    encoding=[]
    for i, sentence in enumerate(sentences):
        # Looking up the mapping dictionary and assigning the index to the respective words
        encoding+=[[word2idx[word] if word in word2idx else unknown_index for word in sentence]]
    return encoding

train_sentences=one_hot(train_seq,word2idx)
val_sentences=one_hot(val_seq,word2idx)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


**Padding input data**

Models in Keras (and elsewhere) take batches of sentences of the same length as input. It is because Deep Learning framework have been designed to handle well Tensors, which are particularly suited for fast computation on the GPU.

Since sentences have different sizes, we "pad" them. That is, we add dummy "padding" tokens so that they all have the same length.

The input to a Keras model thus has this size : (batchsize, maxseqlen) where maxseqlen is the maximum length of a sentence in the batch.

In [0]:
# 2 - Pad your sequences using tf.keras.preprocessing.sequence.pad_sequences
#     https://keras.io/preprocessing/sequence/ 
# Defining a function that either shortens sentences or pads sentences with 0 to a fixed length 
def pad_input(sentences, seq_len,pad_idx=0): 
    features = np.zeros((len(sentences), seq_len),dtype=int)+pad_idx 
    for ii, sentence in enumerate(sentences):
        if len(sentence) != 0:
            features[ii, -len(sentence):] = np.array(sentence)[:seq_len]
    return features
seq_len = 52 #Max length on the set 
train_sentences = pad_input(train_sentences, seq_len)
val_sentences= pad_input(val_sentences, seq_len)

## 4.2 - Design and train your model

In [0]:
# 3 - Design your encoder + classifier using tensorflow.keras.layers
#     In Keras, Torch and other deep learning framework, we create a "container" which is the Sequential() module.
#     Then we add components to this container : the lookup-table, the LSTM, the classifier etc.
#     All of these components are contained in the Sequential() and are trained together.
#     Note that the embedding layer is initialized randomly and does not take advantage of pre-trained word embeddings.

import torch.nn as nn
class Classifier(nn.Module):
    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, drop_prob=0.5):
        super(Classifier, self).__init__()
        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, dropout=drop_prob, batch_first=True)
        #self.fc = nn.Sequential(nn.Dropout(drop_prob),nn.Linear(hidden_dim, output_size))
        
        self.fc = nn.Sequential(nn.Dropout(drop_prob),nn.Linear(hidden_dim,hidden_dim),nn.BatchNorm1d(hidden_dim),
                                nn.Dropout(drop_prob),nn.Linear(hidden_dim,output_size))
        
    def forward(self, x):
        batch_size = x.size(0)
        x = x.long()
        embeds = self.embedding(x)
        lstm_out, self.hidden = self.lstm(embeds, self.hidden)
        lstm_out = lstm_out.contiguous().view( -1,self.hidden_dim)
        #print(lstm_out.shape)
        out = self.fc(lstm_out)
        out=out.reshape(x.size(0),-1,self.output_size)
        return out[:,-1]
    
    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device))
        self.hidden=hidden
        return 


In [38]:
# 4 - Define your loss/optimizer/metrics

is_cuda = torch.cuda.is_available()
if is_cuda:
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

vocab_size = 50000
output_size = 5
embedding_dim = 64
hidden_dim = 32
n_layers = 2

model = Classifier(vocab_size, output_size, embedding_dim, hidden_dim, n_layers)
model.to(device)
print(model)
lr=0.001
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr,weight_decay=1e-3)  

Classifier(
  (embedding): Embedding(50000, 64)
  (lstm): LSTM(64, 32, num_layers=2, batch_first=True, dropout=0.5)
  (fc): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=32, out_features=32, bias=True)
    (2): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Dropout(p=0.5, inplace=False)
    (4): Linear(in_features=32, out_features=5, bias=True)
  )
)


In [39]:
# 5 - Train your model and find the best hyperparameters for your dev set
#     you will be evaluated on the quality of your predictions on the test set
#     Keras expects y_train and y_dev to be one-hot encodings of the labels, i.e. with shape=(n_samples, 5)
import copy
def Train_model(model,criterion,optimizer,loader,device,scheduler=None,mode='train'):
  """Model Training/Validation
  ------------------------------
  Output:
  -----------
  mean_loss: Mean loss over Epoch"""

  if mode=='train':
    model.train()
  elif mode=='val':
    model.eval()
  else:
    print('mode should be Train or Val')
    return
  losses=[]
  Preds=[]
  Actual=[]
  correct = 0
  total = 0
  for inputs, labels in loader:
    inputs, labels = inputs.to(device), labels.long().to(device)
    model.init_hidden(inputs.size(0))
    optimizer.zero_grad()
    with torch.set_grad_enabled(mode=='train'):
      output = model(inputs)
      loss = criterion(output, labels)
      _,pred=torch.max(output, 1)
      if mode=='train':
        loss.backward()
        optimizer.step()
    losses.append(loss.item())
    #Preds=np.append(Preds,pred.cpu().long().numpy())
    #Actual=np.append(Actual,labels.cpu().long().numpy())
    correct+=float(torch.sum(pred==labels.data))
    total+=float(inputs.shape[0])

  if mode =='train' and scheduler!=None:
    scheduler.step()
  mean_loss=np.mean(losses)
  print('{} Loss {:.4f}  Acc : {:.2%}  '.format(mode,mean_loss,correct/total))
  return correct/total
#------------------------------------------------------------------------------------------------------------

train_data = TensorDataset(torch.from_numpy(train_sentences), torch.from_numpy(np.array(y_train)))
val_data= TensorDataset(torch.from_numpy(val_sentences), torch.from_numpy(np.array(y_val)))
batch_size = 64
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
val_loader = DataLoader(val_data, shuffle=True, batch_size=batch_size)

epochs=30
acc_t=[]
acc_v=[]
best_acc=0
for epoch in range(epochs):
  print('epoch: ',epoch)
  acc_t+=[Train_model(model,criterion,optimizer,train_loader,device,mode='train')]
  acc_v+=[Train_model(model,criterion,optimizer,val_loader,device,mode='val')]
  if acc_v[-1]>best_acc:
        best_acc=acc_v[-1]
        best_w=copy.deepcopy(model.state_dict())
print('Best Accuracy: ',best_acc)

epoch:  0
train Loss 1.6257  Acc : 24.50%  
val Loss 1.5726  Acc : 27.25%  
epoch:  1
train Loss 1.5842  Acc : 26.35%  
val Loss 1.5670  Acc : 27.79%  
epoch:  2
train Loss 1.5727  Acc : 27.36%  
val Loss 1.5672  Acc : 27.61%  
epoch:  3
train Loss 1.5732  Acc : 27.06%  
val Loss 1.5725  Acc : 26.88%  
epoch:  4
train Loss 1.5691  Acc : 27.47%  
val Loss 1.5724  Acc : 26.70%  
epoch:  5
train Loss 1.5610  Acc : 27.90%  
val Loss 1.5586  Acc : 28.97%  
epoch:  6
train Loss 1.5529  Acc : 28.99%  
val Loss 1.5462  Acc : 31.43%  
epoch:  7
train Loss 1.5394  Acc : 30.71%  
val Loss 1.5321  Acc : 31.61%  
epoch:  8
train Loss 1.5189  Acc : 32.28%  
val Loss 1.5207  Acc : 33.51%  
epoch:  9
train Loss 1.4958  Acc : 33.49%  
val Loss 1.5081  Acc : 34.15%  
epoch:  10
train Loss 1.4698  Acc : 35.42%  
val Loss 1.4847  Acc : 35.60%  
epoch:  11
train Loss 1.4220  Acc : 38.35%  
val Loss 1.4523  Acc : 36.69%  
epoch:  12
train Loss 1.3676  Acc : 40.64%  
val Loss 1.4320  Acc : 38.69%  
epoch:  1

In [0]:
# 6 - Generate your predictions on the test set using model.predict(x_test)

model.load_state_dict(best_w)
test_sentences=one_hot(test_seq,word2idx)
test_sentences= pad_input(test_sentences, seq_len)
test_data= TensorDataset(torch.from_numpy(test_sentences))
test_loader = DataLoader(test_data, shuffle=False, batch_size=batch_size)

Preds=np.array([],dtype=int)
for inputs in test_loader:
    inputs= inputs[0].to(device)
    model.init_hidden(inputs.size(0))
    with torch.no_grad():
        output = model(inputs)
        _,pred=torch.max(output, 1)
    Preds=np.append(Preds,pred.cpu().numpy())
pred=[str(i)+'\n' for i in Preds]
file = open('logreg_lstm_y_test_sst.txt',"w")
file.writelines(pred)
file.close()

## 4.3 - innovate !

In [32]:
# 7 - Open question: find a model that is better on your dev set
#     (e.g: use a 1D ConvNet, use a better classifier, pretrain your lookup tables ..)
#     you will get point if the results on the test set are better: be careful of not overfitting your dev set too much..
#     Attach the output file "XXX_XXX_y_test_sst.txt" to your deliverable.
#-------------------------------------------------------------------------------------------------------------------
#We'll Use Gensim Library 
from gensim.test.utils import datapath, get_tmpfile 
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec
#-------------------------------------------------------------------------------------------------------------------
#Downloading the publicly available embeddings  to data Directory
!wget  http://nlp.stanford.edu/data/glove.840B.300d.zip -P data 
!unzip data/glove.840B.300d.zip -d data
#-------------------------------------------------------------------------------------------------------------------
#Loading Pretraine d EMbedding 
glove_file = r'data/glove.840B.300d.txt'
#temporary file for word2vec
open('data/word2vec.txt', 'a').close()
tmp_file ='data/word2vec.txt'
#tmp_file = get_tmpfile(r'data/word2vec.txt')
word2vec = glove2word2vec(glove_file, tmp_file)
glove = KeyedVectors.load_word2vec_format(tmp_file)
embed_dim=glove.vectors.shape[1]

#-------------------------------------------------------------------------------------------------------------------
#Recreating Dataset based on Indexes of Pretrained embedding 
#-------------------------------------------------------------------------------------------------------------------

train_s,y_train=create_dataset(train_filepath)
val_s,y_val=create_dataset(dev_filepath)
words = Counter() #Dictionary that will map a word to the number of times it appeared in all the training sentences
train_s=Tokenize(train_s,words)
val_s=Tokenize(val_s)

# Adding padding and unknown to our vocabulary so that they will be assigned an index
extra=['_PAD','_UNK']
words = extra + list(words)

#Creating word2idx for Pretrained Glove EMbeddings
glove.add(extra,[np.zeros(embed_dim),np.random.rand(embed_dim)*2-1])

#Creating Tensor for Pytorch Embedding
weights = torch.FloatTensor(glove.vectors).to(device)

# Dictionaries to store the word to index mappings and vice versa
word2idx={}
for word in words:
    if word in glove: 
        word2idx[word] =glove.vocab[word].index
    else:
        word2idx[word] =glove.vocab['_UNK'].index
        
train_sentences=one_hot(train_s,word2idx,unknown_index=glove.vocab['_UNK'].index)
val_sentences=one_hot(val_s,word2idx,unknown_index=glove.vocab['_UNK'].index)
train_sentences = pad_input(train_sentences, seq_len,pad_idx=glove.vocab['_PAD'].index)
val_sentences= pad_input(val_sentences, seq_len,pad_idx=glove.vocab['_PAD'].index)



#Wrappers for Pytorch Data Loaders
train_data = TensorDataset(torch.from_numpy(train_sentences), torch.from_numpy(np.array(y_train)))
val_data= TensorDataset(torch.from_numpy(val_sentences), torch.from_numpy(np.array(y_val)))
batch_size = 64
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
val_loader = DataLoader(val_data, shuffle=True, batch_size=batch_size)

#Model -------------------------------------------------------------------------------------------------------------------
class Classifier(nn.Module):
    def __init__(self, output_size, hidden_dim, n_layers,weights, drop_prob=0.5,embed_dim=50):
        super(Classifier, self).__init__()
        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        self.embedding =nn.Embedding.from_pretrained(weights)
        self.lstm = nn.LSTM(embed_dim, hidden_dim, n_layers, dropout=drop_prob, batch_first=True)
        self.fc = nn.Sequential(nn.Linear(hidden_dim, output_size))
    
    def forward(self, x):
        batch_size = x.size(0)
        x = x.long()
        
        embeds = self.embedding(x)
        #print(embeds.shape)
        lstm_out, self.hidden = self.lstm(embeds, self.hidden)
        lstm_out = lstm_out.contiguous().view( -1,self.hidden_dim)
        #print(lstm_out.shape)
        out = self.fc(lstm_out)
        out=out.reshape(x.size(0),-1,self.output_size)
        return out[:,-1]
    
    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device))
        self.hidden=hidden
        return 
#-------------------------------------------------------------------------------------------------------------------
output_size = 5
hidden_dim = 64
n_layers = 2
model = Classifier(output_size, hidden_dim, n_layers,weights,embed_dim=embed_dim)
model.to(device)
print(model)
lr=0.0005
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr,weight_decay=1e-3)  

#Freezing embedding Weights : No grad
model.embedding.weight.requires_grad=False
#Training Model
epochs=30
acc_t=[]
acc_v=[]
best_acc=0
for epoch in range(epochs):
  print('epoch: ',epoch)
  acc_t+=[Train_model(model,criterion,optimizer,train_loader,device,mode='train')]
  acc_v+=[Train_model(model,criterion,optimizer,val_loader,device,mode='val')]
  if acc_v[-1]>best_acc:
        best_acc=acc_v[-1]
        best_w=copy.deepcopy(model.state_dict())
print('best accuracy on Validation set',best_acc)
#-------------------------------------------------------------------------------------------------------------------
#Create Prediction For test set
model.load_state_dict(best_w)
test_s=create_dataset(test_filepath,test=True)
test_s=Tokenize(test_s)
test_sentences=one_hot(test_s,word2idx,unknown_index=glove.vocab['_UNK'].index)
test_sentences= pad_input(test_sentences, seq_len,pad_idx=glove.vocab['_PAD'].index)
test_data= TensorDataset(torch.from_numpy(test_sentences))
test_loader = DataLoader(test_data, shuffle=False, batch_size=batch_size)

Preds=np.array([],dtype=int)
for inputs in test_loader:
    inputs= inputs[0].to(device)
    model.init_hidden(inputs.size(0))
    with torch.no_grad():
        output = model(inputs)
        _,pred=torch.max(output, 1)
    Preds=np.append(Preds,pred.cpu().numpy())
pred=[str(i)+'\n' for i in Preds]
file = open('XXX_XXX_y_test_sst.txt',"w")
file.writelines(pred)
file.close()

Classifier(
  (embedding): Embedding(2196018, 300)
  (lstm): LSTM(300, 64, num_layers=2, batch_first=True, dropout=0.5)
  (fc): Sequential(
    (0): Linear(in_features=64, out_features=5, bias=True)
  )
)
epoch:  0
train Loss 1.5210  Acc : 31.65%  
val Loss 1.3878  Acc : 36.78%  
epoch:  1
train Loss 1.3563  Acc : 40.58%  
val Loss 1.3383  Acc : 39.87%  
epoch:  2
train Loss 1.3128  Acc : 42.04%  
val Loss 1.3206  Acc : 40.42%  
epoch:  3
train Loss 1.2752  Acc : 43.49%  
val Loss 1.3164  Acc : 40.33%  
epoch:  4
train Loss 1.2528  Acc : 44.73%  
val Loss 1.3453  Acc : 41.14%  
epoch:  5
train Loss 1.2358  Acc : 46.02%  
val Loss 1.3001  Acc : 42.51%  
epoch:  6
train Loss 1.2148  Acc : 47.17%  
val Loss 1.2767  Acc : 43.60%  
epoch:  7
train Loss 1.2028  Acc : 47.34%  
val Loss 1.3198  Acc : 42.14%  
epoch:  8
train Loss 1.1893  Acc : 48.34%  
val Loss 1.2892  Acc : 42.78%  
epoch:  9
train Loss 1.1742  Acc : 48.67%  
val Loss 1.2887  Acc : 43.87%  
epoch:  10
train Loss 1.1686  Acc :